# Prepare metabolomics data for ipath ingestion
This notebook is dedicated to preparing the VBCF metabolomics facility results for upload to ipath. However, since the data is provided as excel file containing several sheets one needs to do some preprocessing in excel in order to make the data parsable. In particular these preprocessing/reformating steps apply only to the 'Results HILIC' and 'Results RP' sheets since these are the main data sources.

1. Reformat the log2 fold change cells to contain numbers with at least 3 decimal places
2. Reformat the normalized peak area cells to contain numbers with at least 10 decimal places
3. Save each sheet as UTF-8 decoded CSV file

The rest will be done by `parse_metabolomics_results` function which will return two `pandas.DataFrame` objects containing all the data you need (one containing everything that has a KEGG ID and one containing everything without a KEGG ID)

In [33]:
import pandas as pd
import csv
import requests
def compounds_to_kegg(compounds):
    url = "http://api.xialab.ca/mapcompounds"

    payload = '{\n\t"queryList": "' + ';'.join(compounds) + ';",\n\t"inputType": "name"\n}'
    headers = {
        'Content-Type': "application/json",
        'cache-control': "no-cache",
    }

    response = requests.request("POST", url, data=payload, headers=headers)

    return response.text

def get_unambiguous_column_names(column_names):
    series = pd.Series(column_names)
    series[series.duplicated(keep = 'first')] = series[series.duplicated(keep = 'first')].apply(
        lambda x: x + '.1'
    )
    return series.to_list()

def get_ipath_selection(df, identifier, color, width, print_selection = True):
    selection = []
    for i in df[identifier]:
        entry = [i, color, f'W{str(width)}']
        selection.append(entry)
        
        if print_selection:
            print(' '.join(entry))

def parse_metabolomics_results(filename, delimiter = ',', quotechar = '"'):
    with open(filename) as csvfile:
        data = []
        entryid = None
        csvreader = csv.reader(
            csvfile,
            delimiter = delimiter,
            quotechar = quotechar
        )
        
        # get column names and make them unambiguous
        names = []
        for i in range(2):
            names = csvreader.__next__()
        
        names = get_unambiguous_column_names(names)
        
        add_names = [
                'Name', 
                'Molecular Weight', 
                'RT [min]', 
                'DeltaMass [ppm]', 
                'DBID', 
                'Reference List Name', 
                'mzLogic Score', 
                'ChemSpider ID', 
                'KEGG', 'HMDB', 
                'Mass List Search Results ID'
        ]
        add_names_set = set(add_names) # faster check
        series = None
        tmp_names = []
        for line in csvreader:
            if line[0]:
                entryid = line[0]
                series = pd.Series(
                    {k: v for k, v in zip(names, line)}
                )

            elif line[1] == 'Name':
                tmp_names = [name for name in line if name]
            
            elif line[1] == series.Name:
                for k, v in zip(
                    tmp_names,
                    line[1: 1 + len(tmp_names)]
                ):
                    if k in add_names_set:
                        k = k + '2' if k in {'Name', 'RT [min]'} else k
                        series[k] = v if v else None
                    
                    elif k == 'KEGG ID':
                        series['KEGG'] = v if v else None
                    
                data.append(series.copy())
    
    return pd.DataFrame(data, columns = names + add_names)

def clean_dataframe(df):
    kegg_rows = df.KEGG.isna()
    no_kegg = df.loc[kegg_rows, :].copy()
    df = df.loc[~kegg_rows, :].copy()
    df = df.drop_duplicates(subset = ['ID', 'KEGG'])
    return df, no_kegg

In [34]:
hilic = parse_metabolomics_results('../../Downloads/Results_Untargeted_Metabolomics_E14-P40_LK_Nova_results_HILIC.csv')
print(hilic.ID.unique().shape)
hilic, no_kegg_hilic = clean_dataframe(hilic)
hilic

(372,)


,ID,Name,Calc. MW,RT [min],"(P40, KO) / (P40, WT)","(P2, KO) / (P2, WT)","(E14_5, KO) / (E14_5, WT)","(P40, WT) / (P2, WT)","(P40, WT) / (E14_5, WT)","(P2, WT) / (E14_5, WT)",...,Molecular Weight,RT [min],DeltaMass [ppm],DBID,Reference List Name,mzLogic Score,ChemSpider ID,KEGG,HMDB,Mass List Search Results ID
0,A001_HILIC,Pyruvic acid,88.0160,3.93,-1.32,-0.31,-0.55,0.03,-0.27,-0.30,...,88.0160,3.93,-0.11,B003,iHILIC_neg_2020,68.35,1031,C00022,HMDB0000243,101
1,A002_HILIC,L-(+)-Alanine,89.0477,14.83,0.00,-0.05,-0.04,0.23,-0.34,-0.57,...,89.0477,14.83,0.34,A072,iHILIC_neg_2020,74.93,5735,C00041,HMDB0000161,57
3,A003_HILIC,Sarcosine,89.0477,13.63,0.31,0.04,0.22,-4.07,-4.17,-0.09,...,89.0477,13.63,0.71,A026,iHILIC_pos_2020,36.31,1057,C00213,HMDB0000271,26
5,A004_HILIC,L-(+)-Lactic acid,90.0317,4.48,1.08,0.21,-0.22,-0.28,-0.78,-0.49,...,90.0317,4.48,-0.10,B002,iHILIC_neg_2020,93.20,96860,C00186,HMDB0000190,109
6,A005_HILIC,Glycerin,92.0474,7.10,0.21,-0.06,-0.20,1.64,1.58,-0.06,...,92.0473,7.10,0.17,D034,iHILIC_neg_2020,None,733,C00116,HMDB0000131,473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,B180_HILIC,"α,α-Trehalose",342.1162,15.63,1.90,0.60,0.07,-0.95,-3.74,-2.79,...,342.1162,15.63,-0.19,NaN,NaN,NaN,NaN,C01083,NaN,NaN
237,B183_HILIC,Uridine 5'-diphosphate (UDP),404.0022,18.51,-0.03,-0.04,-0.03,-2.59,-5.55,-2.96,...,404.0022,18.51,0.03,NaN,NaN,NaN,NaN,C00015,NaN,NaN
238,B184_HILIC,Adenosine diphosphate (ADP),427.0296,17.32,-0.17,-0.19,-0.04,-0.58,-2.30,-1.72,...,427.0294,17.32,0.31,NaN,NaN,NaN,NaN,C00008,NaN,NaN
240,B186_HILIC,Uridine 5'-diphosphoglucuronic acid,580.0345,20.93,0.12,-0.19,0.01,-1.59,-2.12,-0.54,...,580.0343,20.93,0.28,NaN,NaN,NaN,NaN,C00167,NaN,NaN


In [20]:
no_kegg_hilic

,ID,Name,Calc. MW,RT [min],"(P40, KO) / (P40, WT)","(P2, KO) / (P2, WT)","(E14_5, KO) / (E14_5, WT)","(P40, WT) / (P2, WT)","(P40, WT) / (E14_5, WT)","(P2, WT) / (E14_5, WT)",...,Molecular Weight,RT [min],DeltaMass [ppm],DBID,Reference List Name,mzLogic Score,ChemSpider ID,KEGG,HMDB,Mass List Search Results ID
50,A033_HILIC,6-Oxo-pipecolinic acid,143.0583,3.85,0.16,0.77,-0.19,-1.59,-4.09,-2.50,...,143.0582,3.85,0.48,A111,iHILIC_neg_2020,39.46,2282737,None,HMDB0061705,578
180,B123_HILIC,2-Hydroxycaproic acid,132.0786,2.50,0.50,-0.25,-0.25,-1.65,-1.87,-0.21,...,132.0786,2.50,-0.42,NaN,NaN,NaN,NaN,None,NaN,NaN
183,B127_HILIC,6-Aminonicotinic acid,138.0430,6.88,-0.68,-0.42,-0.42,0.48,-0.62,-1.10,...,138.0429,6.88,0.74,NaN,NaN,NaN,NaN,None,NaN,NaN
186,B130_HILIC,DL-Stachydrine,143.0947,9.20,0.18,-0.41,-0.31,1.01,-3.60,-4.61,...,143.0946,9.20,0.46,NaN,NaN,NaN,NaN,None,NaN,NaN
194,B139_HILIC,1-Methylguanine,165.0652,11.62,0.32,0.24,0.39,0.25,1.05,0.80,...,165.0651,11.62,0.72,NaN,NaN,NaN,NaN,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,c392_HILIC,PS(16:1(9Z)/18:1(9Z)),759.5050,2.13,0.63,0.14,-0.18,-0.81,-3.47,-2.66,...,759.5050,2.13,0.01,NaN,NaN,None,NaN,NaN,NaN,NaN
444,c393_HILIC,"PS(18:2(9Z,12Z)/18:2(9Z,12Z))",783.5051,2.12,0.78,0.10,-0.09,-2.19,-3.06,-0.87,...,783.5050,2.12,0.02,NaN,NaN,None,NaN,NaN,NaN,NaN
445,c394_HILIC,Phosphatidylserine,791.5676,1.98,0.95,0.84,-0.63,1.50,-1.48,-2.98,...,791.5676,1.98,-0.09,NaN,NaN,None,NaN,NaN,NaN,NaN
446,c395_HILIC,1-oleoyl-2-arachidonoyl-sn-glycero-3-phospho-L...,809.5200,2.11,0.80,0.06,-0.06,-0.69,-1.70,-1.02,...,809.5207,2.11,-0.86,NaN,NaN,None,NaN,NaN,NaN,NaN


In [29]:
import itertools as it
conditions = ['WT', 'KO']
timepoints = ['E14_5', 'P2', 'P40']
def write_raw_values(df, timepoints, conditions, prefix):
    for sample in it.product(timepoints, conditions):
        sample_cols = hilic.columns[
            df.columns.str.match(
                '^S[0-9]{2}_' + '_'.join(sample)
            )
        ].to_list()

        df.loc[:, ['KEGG'] + sample_cols].to_csv(
            '_'.join([prefix, *sample]) + '.tsv',
            sep = '\t',
            header = False,
            index = False
        )

columns_wt = ['(P2, WT) / (E14_5, WT)', '(P40, WT) / (P2, WT)']
columns_ko = ['(P2, KO) / (E14_5, KO)', '(P40, KO) / (P2, KO)']
def write_foldchanges(df, columns, condition, prefix):
    # writing base values for t0
    tmp = pd.DataFrame(
        [['KEGG', 0] for kegg in df.KEGG]
    )
    tmp.to_csv(
        '_'.join([prefix, 'base', condition]) + '.tsv',
        sep = '\t',
        header = False,
        index = False
    )
    
    for column in columns:
        suffix = column[1:-1] \
            .replace(',', '_') \
            .replace(') / (', '_')
        
        df.loc[:, ['KEGG', column]].to_csv(
            '_'.join([prefix, suffix]) + '.tsv',
            sep = '\t',
            header = False,
            index = False
        )
    
# write_raw_values(
#     hilic,
#     timepoints,
#     conditions,
#     '../raw/metabolomics_hilic'
# )
for condition, columns in zip(
    conditions, 
    [columns_wt, columns_ko]
):
    write_foldchanges(
        hilic,
        columns,
        condition,
        '../raw/metabolomics_hilic'
    )
    
selection = get_ipath_selection(hilic, 'KEGG', '#000000', 20)

In [27]:
rp = parse_metabolomics_results('../Downloads/Results_Untargeted_Metabolomics_E14-P40_LK_Nova_results_RP.csv')
print(rp.ID.unique().shape)
rp, no_kegg_rp = clean_dataframe(rp)
rp

(206,)


,ID,Name,Calc. MW,RT [min],"(P40, KO) / (P40, WT)","(P2, KO) / (P2, WT)","(E14_5, KO) / (E14_5, WT)","(P40, WT) / (P2, WT)","(P40, WT) / (E14_5, WT)","(P2, WT) / (E14_5, WT)",...,Molecular Weight,RT [min] 2,DeltaMass [ppm],DBID,Reference List Name,mzLogic Score,ChemSpider ID,KEGG,HMDB,Mass List Search Results ID
0,A001_RP,L-(+)-Alanine,89.0478,3.86,1.000,1.000,1.000,1.000,0.000,0.000,...,89.0477,NaN,1.09,A072,RP_pos_2020,93.42507112,5735,C00041,HMDB0000161,71
1,A002_RP,L-a-Amino-n-butyric acid,103.0633,4.05,1.000,1.000,1.000,0.000,0.000,0.000,...,103.0633,NaN,-0.17,E19,RP_pos_2020,89.90492779,72524,C02356,HMDB0000452,176
2,A003_RP,Choline,103.0997,3.82,1.000,1.000,1.000,0.222,1.000,0.679,...,103.0997,NaN,-0.15,B018,RP_pos_2020,None,299,C00114,HMDB0000097,72
3,A004_RP,L-Serine,105.0426,3.74,1.000,1.000,1.000,0.066,1.000,0.246,...,105.0426,NaN,0.35,A075,RP_neg_2020,91.03202549,5736,C00065,HMDB0000187,19
4,A005_RP,Hypotaurine,109.0198,3.86,1.000,1.000,1.000,0.000,0.000,0.000,...,109.0198,NaN,0.68,B019,RP_pos_2020,70.9566742,96959,C00519,HMDB0000965,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,B106_RP,Palmitoylcarnitine,399.3350,27.42,1.000,1.000,1.000,0.000,0.013,0.266,...,399.3349,NaN,0.38,NaN,NaN,NaN,NaN,,NaN,NaN
151,B107_RP,Adenosine diphosphate (ADP),427.0298,4.74,1.000,1.000,1.000,0.781,0.000,0.000,...,427.0294,NaN,0.78,NaN,NaN,NaN,NaN,C00008,NaN,NaN
152,B108_RP,Glycerophospho-N-palmitoyl ethanolamine,453.2857,30.07,0.095,1.000,1.000,0.002,0.000,0.876,...,453.2855,NaN,0.38,NaN,NaN,NaN,NaN,,NaN,NaN
153,B109_RP,Adenosine diphosphate ribose,559.0720,4.72,1.000,1.000,1.000,0.002,0.000,0.414,...,559.0717,NaN,0.51,NaN,NaN,NaN,NaN,,NaN,NaN
